In [1]:
import torch

In [2]:
def reg(x):
    # l2 = torch.linalg.vector_norm(x[:, -1], ord=2, dim=0)
    # l2 =  torch.linalg.vector_norm(x, ord=2, dim=0, keepdim=False)[-1]
    # l2_normalized = l2 / x.size(0)
    l2 = torch.mean(x[:, -1] ** 2)
    return l2 #TODO: we could divide by the squared number of elements to be more invariant to the number of Gaussians..

In [3]:
x1 = torch.ones(10, 3)

x2 = torch.ones(10, 3)
x2[:, 2] = x1[:, 2] * 50

x3 = torch.ones(20, 3)
x3[:, 2] = x3[:, 2] * 50


assert reg(x1) < reg(x2) 

# assert reg same, otherwise show diff
assert reg(x2) == reg(x3), abs(reg(x2) - reg(x3))


reg(x1), reg(x2), reg(x3)


(tensor(1.), tensor(2500.), tensor(2500.))

In [4]:
from sisa3d.camera import convert_png_to_jpeg

ImportError: cannot import name 'convert_png_to_jpeg' from 'sisa3d.camera' (/Users/julius/repos/master-thesis-ds/ptdraft/sisa3d/camera.py)

In [ ]:
input_path = "/Users/julius/repos/master-thesis-ds/results/preprocessing/blueberry_rgba.png"
convert_png_to_jpeg(input_path, "results/preprocessing/blueberry_rgba.jpg")

In [ ]:
from PIL import Image

im = Image.open(input_path)
rgb_im = im.convert('RGB')
rgb_im.save('tmpt.jpg')

In [ ]:
from sisa3d.clip import compute_clip

path = "/Users/julius/repos/master-thesis-ds/results/dream_gaussian/stage_1/generated"
path1 = "/Users/julius/repos/master-thesis-ds/results/dream_gaussian/stage_1/reference"

compute_clip(path, path1, id = "test", csv_file="results/blueberry_rgba_clip.csv", average_scores=False)

Calculating individual CLIP Scores for each pair:


100%|██████████| 1/1 [00:07<00:00,  7.77s/it]

Scores written to results/blueberry_rgba_clip.csv for ID: test


In [ ]:
import yaml
import itertools
import subprocess
import copy

# Load the default configuration
with open('../config.yaml', 'r') as file:
    default_config = yaml.safe_load(file)

# Define the grid of parameters to test
param_grid = {
    'training_batch_size': [1, 2, 4],
    'training_lambda_zero123': [0, 0.5, 1],
    'input_density_thresh': [0.5, 1, 1.5]
}

# Function to update nested dictionaries
def update_config(base, changes):
    for key, value in changes.items():
        if isinstance(value, dict) and key in base:
            update_config(base[key], value)
        else:
            base[key] = value

# Iterate over all combinations of grid parameters
for values in itertools.product(*param_grid.values()):
    # Create a temporary config for this combination
    temp_config = copy.deepcopy(default_config)
    config_updates = dict(zip(param_grid.keys(), values))
    
    # Convert flat keys to nested dictionary updates
    nested_updates = {}
    for key, value in config_updates.items():
        keys = key.split('_')
        current = nested_updates
        for part in keys[:-1]:
            if part not in current:
                current[part] = {}
            current = current[part]
        current[keys[-1]] = value

    # Update the configuration
    update_config(temp_config, nested_updates)
    
    # Save the modified configuration to a temporary YAML file
    temp_yaml_path = 'temp_config.yaml'
    with open(temp_yaml_path, 'w') as file:
        yaml.dump(temp_config, file)

In [ ]:
import yaml
import itertools
import os
from tempfile import NamedTemporaryFile

import yaml
import itertools
import os

def create_tmp_config_files(default_config_path, grid_search_config_path, output_dir='configs'):
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Load the default configuration
    with open(default_config_path, 'r') as file:
        default_config = yaml.safe_load(file)
    
    # Load the grid search parameters
    with open(grid_search_config_path, 'r') as file:
        grid_search_config = yaml.safe_load(file)
    
    # Extract the grid search parameters and fixed values
    grids = grid_search_config.get('grid_search', {})
    fixed_values = grid_search_config.get('fixed_values', {})
    
    # Update the default config with any fixed values first
    default_config.update(fixed_values)

    # Generate all combinations of grid search parameters
    keys, values = zip(*grids.items())
    for i, value_combination in enumerate(itertools.product(*[itertools.product([key], val) for key, val in values])):
        # Create a temporary dictionary with updated values from one combination
        temp_config = default_config.copy()
        for key, val in value_combination:
            nested_keys = key.split('.')
            sub_config = temp_config
            for sub_key in nested_keys[:-1]:
                sub_config = sub_config.setdefault(sub_key, {})
            sub_config[nested_keys[-1]] = val
        
        # Create a file to save this configuration
        temp_filename = os.path.join(output_dir, f'tmp_config_{i}.yaml')
        with open(temp_filename, 'w') as tmp_file:
            yaml.safe_dump(temp_config, tmp_file, default_flow_style=False)
            print(f"Created config file: {temp_filename}")

# Example usage
# create_tmp_config_files('default_config.yaml', 'grid_search_config.yaml')


# Example usage
create_tmp_config_files('../config.yaml', '../grid_search.yaml')


Created config file: configs/tmp_config_0.yaml
Created config file: configs/tmp_config_1.yaml
Created config file: configs/tmp_config_2.yaml
Created config file: configs/tmp_config_3.yaml
Created config file: configs/tmp_config_4.yaml
Created config file: configs/tmp_config_5.yaml
Created config file: configs/tmp_config_6.yaml
Created config file: configs/tmp_config_7.yaml
Created config file: configs/tmp_config_8.yaml
Created config file: configs/tmp_config_9.yaml
Created config file: configs/tmp_config_10.yaml
Created config file: configs/tmp_config_11.yaml
Created config file: configs/tmp_config_12.yaml
Created config file: configs/tmp_config_13.yaml
Created config file: configs/tmp_config_14.yaml
Created config file: configs/tmp_config_15.yaml
Created config file: configs/tmp_config_16.yaml
Created config file: configs/tmp_config_17.yaml
Created config file: configs/tmp_config_18.yaml
Created config file: configs/tmp_config_19.yaml
Created config file: configs/tmp_config_20.yaml
Cr

KeyboardInterrupt: 

Generated 36 configuration files in config_files.


In [6]:
from sisa3d.gridsearch import create_grid_search_config_files

In [7]:
create_grid_search_config_files('../config.yaml', '../grid_search.yaml', "../gridsearch_configs")

Generated 36 configuration files in ../gridsearch_configs.
